In [6]:
hint = hexstr_to_bytes("5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e")
print(f'[magenta]{hint.hex()}, [bold cyan]{hint.hex()}')

5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e, 
5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e

In [9]:
full_node_client = await get_full_node_client()
hint_cr = await full_node_client.get_coin_records_by_hint(hint)
await close_rpc_client(full_node_client)

[
    CoinRecord(
        coin=Coin { parent_coin_info: 86aff79387f8ae0ca020cca5267662c066560e8f81e4e6d5caa166751017632f, puzzle_hash: 1348c1c36b7e723207a669d0a2c06dce2120306957dee2338f71ef6d448ee61a, amount: 1 },
        confirmed_block_index=6,
        spent_block_index=8,
        coinbase=False,
        timestamp=1668611302
    ),
    CoinRecord(
        coin=Coin { parent_coin_info: 552add06fcb8986371bc3892233a3d29021377220411a072766486f005d7b42c, puzzle_hash: 1348c1c36b7e723207a669d0a2c06dce2120306957dee2338f71ef6d448ee61a, amount: 1 },
        confirmed_block_index=8,
        spent_block_index=0,
        coinbase=False,
        timestamp=1668611718
    )
]

In [20]:
from rich.console import Console
from rich.table import Table
from rich import box

console = Console()

table = Table(show_header=True, header_style="bold magenta", box=box.ASCII)
table.add_column("Coin Name", style="dim")
table.add_column("Spent")
for cr in hint_cr:
    coin_name = cr.coin.name().hex()
    spent = "True" if cr.spent_block_index != 0 else "False"
    table.add_row(
        coin_name, spent
    )
console.print(table)

+--------------------------------------------------------------------------+
| Coin Name                                                        | Spent |
|------------------------------------------------------------------+-------|
| 552add06fcb8986371bc3892233a3d29021377220411a072766486f005d7b42c | True  |
| 4c57566e626efb6f857b80e3d29cc3082b8712340480ddfd7370379cfb3a6a95 | False |
+--------------------------------------------------------------------------+

In [38]:
from rich.tree import Tree
from rich import print

tree = Tree(f'[bold green]{hint.hex()}')
for cr in hint_cr:
    spent = cr.spent_block_index != 0
    name_color = "red" if spent else "cyan"
    coin_name = cr.coin.name().hex()
    coin_tree = tree.add(f'[{name_color}]{coin_name}')

    coin_tree.add(cr.coin.parent_coin_info.hex())
    coin_tree.add(cr.coin.puzzle_hash.hex())
    coin_tree.add(f'{cr.coin.amount}')

print(tree)

5cd10e3c7229f0ac3fde1b6cdb99650fe4b120bb028662464effb45facd9ac7e
├── 552add06fcb8986371bc3892233a3d29021377220411a072766486f005d7b42c
│   ├── 86aff79387f8ae0ca020cca5267662c066560e8f81e4e6d5caa166751017632f
│   ├── 1348c1c36b7e723207a669d0a2c06dce2120306957dee2338f71ef6d448ee61a
│   └── 1
└── 4c57566e626efb6f857b80e3d29cc3082b8712340480ddfd7370379cfb3a6a95
    ├── 552add06fcb8986371bc3892233a3d29021377220411a072766486f005d7b42c
    ├── 1348c1c36b7e723207a669d0a2c06dce2120306957dee2338f71ef6d448ee61a
    └── 1

In [47]:
clsp = '''
(   
    (defconstant EMPTY ' ')
    (defconstant MIN_POS 0)
    (defconstant MAX_POS 8)
    ;  o | o | x 
    ; ---+---+---
    ;  x | x | x 
    ; ---+---+---
    ;  o | x | o 
    (defun get-new-board-rec (board v count pos)
        (if (l board)
            (c
                (if (= count pos) 
                    (if (= EMPTY (f board))
                        v
                        (x pos "position already occupied")
                    )
                    (f board)
                ) 
                (get-new-board-rec 
                    (r board) v (+ count 1) pos
                )
            )
            () 
        )
    )
'''
from rich.console import Console
from rich.syntax import Syntax

console = Console()
syntax = Syntax(clsp, "clojure", line_numbers=True)
console.print(syntax)

   1                                                                                                               
   2 (                                                                                                             
   3     (defconstant EMPTY ' ')                                                                                   
   4     (defconstant MIN_POS 0)                                                                                   
   5     (defconstant MAX_POS 8)                                                                                   
   6     ;  o | o | x                                                                                              
   7     ; ---+---+---                                                                                             
   8     ;  x | x | x                                                                                              
   9     ; ---+---+---                                                                                             
  10     ;  o | x | o                                                                                              
  11     (defun get-new-board-rec (board v count pos)                                                              
  12         (if (l board)                                                                                         
  13             (c                                                                                                
  14                 (if (= count pos)                                                                             
  15                     (if (= EMPTY (f board))                                                                   
  16                         v                                                                                     
  17                         (x pos "position already occupied")                                                   
  18                     )                                                                                         
  19                     (f board)                                                                                 
  20                 )                                                                                             
  21                 (get-new-board-rec                                                                            
  22                     (r board) v (+ count 1) pos                                                               
  23                 )                                                                                             
  24             )                                                                                                 
  25             ()                                                                                                
  26         )                                                                                                     
  27     )                                                                                                         
  28                                                                                                               